In [1]:
import io
import csv
import os.path

import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt

%matplotlib qt5

In [2]:
import sys

In [3]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
from posarmctools.ekfnavtools import *

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.PosarMCParameters import *

In [5]:
%load_ext autoreload
%autoreload 2

## Load navigation data SbgLogEkfNav

In [6]:
prefix = "/home/pleroy/DATA/PoSAR-v2_PIMA_TEST-2/dataLogger/session_0002/2018_04_23/13h00/"

logEkfEuler_13h00 = prefix+ "sbgLogEkfEuler.dat"
logEkfNav_13h00  = prefix + "sbgLogEkfNav.dat"
logUtcData_13h00 = prefix + "sbgLogUtcData.dat"
logGpsPos_13h00 = prefix + "sbgLogGpsPos.csv"
logGpsVel_13h00 = prefix + "sbgLogGpsVel.dat"

### Load GPS Pos logs

In [7]:
idx_gpsPos_lat = 3
idx_gpsPos_long = 4
idx_gpsPos_alt = 5
idx_gps_lat = 1
idx_gps_long = 2
idx_gps_alt = 3

gps_13h00 = np.loadtxt( logGpsPos_13h00, skiprows = 1, delimiter = ',',
                 usecols = (0, idx_gpsPos_lat, idx_gpsPos_long, idx_gpsPos_alt) )

gps = gps_13h00

### Load GPS Vel logs

In [8]:
idx_gpsVel_north = 3
idx_gpsVel_east = 4
idx_gpsVel_down = 5
idx_gpsVel_course = 9

vel_13h00 = np.loadtxt( logGpsVel_13h00, skiprows = 1, 
                 usecols = (0, idx_gpsVel_north, idx_gpsVel_east, idx_gpsVel_down, idx_gpsVel_course) )

vel = vel_13h00

In [9]:
Lat = gps[:,idx_gps_lat]
Long = gps[:,idx_gps_long]
Alt = gps[:,idx_gps_alt]
Vel = ( vel[:,1]**2 + vel[:,2]**2 + vel[:,3]**2) **0.5
course = vel[:,4]

### Load UTC data logs

In [10]:
#timeStamp status year month day hour minute second nanoSecond gpsTimeOfWeek
idx_h = 5
idx_m = 6
idx_s = 7
idx_nano = 8

utc_13h00 = np.loadtxt( logUtcData_13h00, skiprows = 1, usecols = (0, idx_h, idx_m, idx_s, idx_nano) )

utc = utc_13h00

In [11]:
printUtc( 0, utc )
printUtc( -1, utc )

12.0 : 59.0 : 59.995
13.0 : 59.0 : 59.990


## Plot navigation data jointly with track references

In [12]:
plt.figure()
ax = plt.subplot(111)
plotLongLatAndTrackReferences( ax, Long, Lat )
plotRunaway( ax )
plt.title( "GPS data and track references\n" + prefix )
plt.grid()

## Check the existence of the record files

In [13]:
data_date = "2018_04_23_15_33_03"
data_dir = "/home/pleroy/DATA/PoSAR-v2_PIMA_TEST-2/" + data_date
firstRecord = 0
lastRecord = 140

In [14]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters_v2( params_filename )

In [15]:
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
rampsPerBuffer = params.rampsPerBuffer
samplesPerFile = params.samplesPerRamp * params.rampsPerFile
buffersPerFile = params.buffersPerFile
T_files = samplesPerFile / 10e6
nbRecords = int((lastRecord - firstRecord) / params.buffersPerFile + 1)
print( "nbRecords = {}".format(nbRecords) )

nbRecords = 71


In [16]:
fileList = []

for k in range(firstRecord, lastRecord, buffersPerFile):
    filename = data_dir + "/record" + str(k) + ".bin"
    ret = os.path.isfile(filename)
    if ret == False:
        fileList.append(0)
    else:
        fileList.append(1)

idxFileList = np.arange( len( fileList ) ) * buffersPerFile + firstRecord

In [17]:
plt.figure()
plt.plot(idxFileList, fileList, 'o-')
plt.grid()

### Process timeStamps.data

In [18]:
timeStampsFile = data_dir + "/" + data_date + "_timeStamps.data"
bufferNumber_0, timeStamp_0 = np.loadtxt( timeStampsFile, skiprows = 1, unpack=True )
rampNumber_0 = bufferNumber_0 * rampsPerBuffer

#====================================================================
# the last value of the file may be erroneous, this should be checked
#====================================================================

#===================================================================
# the first value may be erroneous, one may have to shift all values
#===================================================================

timeStamp_1 = timeStamp_0
bufferNumber_1 = bufferNumber_0
rampNumber_1 = bufferNumber_1 * rampsPerBuffer

In [19]:
plt.figure()
plt.subplot(211)
plt.plot( timeStamp_1, ".", label=data_date )
plt.grid()
plt.legend()
plt.title("timeStamp wrt fileNumber / " + data_date)
plt.subplot(212)
plt.plot( np.diff(timeStamp_1)/1e6, "." )
plt.grid()
plt.title( "diff(timeStamp) wrt fileNumber / " + data_date )

**Be carefull here, there could be a jump in the timeStamp values due to the counter saturation at 2^32. The mitigation is following.**

In [ ]:
timeStamp_1[np.where(timeStamp_1 < 2**31)] +=  2**32
gps[np.where(gps[:, 0] < 2**31),0] += 2**32
vel[np.where(vel[:, 0] < 2**31),0] += 2**32

In [ ]:
plt.figure()
plt.plot( gps[:, 0] )

In [ ]:
# interpolate GPS data for each record
timeStamp_records = timeStamp_1
Lat_records = np.interp( timeStamp_records, gps[:,0], Lat )
Long_records = np.interp( timeStamp_records, gps[:,0], Long )
Alt_records = np.interp( timeStamp_records, gps[:,0], Alt )
Vel_records = np.interp( timeStamp_records, vel[:,0], Vel )
course_records = np.interp( timeStamp_records, vel[:,0], course )

### Data selection for focalization

In [ ]:
firstFile = 0
nbFiles = 71
lastFile = firstFile + nbFiles - 1
fileRange = np.arange(firstFile, lastFile + 1)
timeStamp_selection = timeStamp_1[fileRange]

In [ ]:
# interpolate GPS data for each file of the selection
Lat_selection = np.interp( timeStamp_selection, gps[:,0], gps[:,1] )
Long_selection = np.interp( timeStamp_selection, gps[:,0], gps[:,2] )
Alt_selection = np.interp( timeStamp_selection, gps[:,0], gps[:,3] )
Vel_selection = np.interp( timeStamp_selection, vel[:,0], Vel )
course_selection = np.interp( timeStamp_selection, vel[:,0], course )

### Plot navigation data and record periods

In [ ]:
plt.figure()

ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax, Long, Lat )
plotRunaway( ax )

plt.plot( Long_records, Lat_records, "Dg", markeredgecolor='black', 
         label=data_date )
plt.plot( Long_selection, Lat_selection, "Dy", markeredgecolor='black', 
         label="files selected for focalization {} to {}".format(firstFile, lastFile) )

title = "Navigation data-track references-record locations-" + data_date
plt.title("Navigation data, track references, record locations\n" + data_date)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.grid()

plt.savefig( data_dir + "/" + title + ".png", bbox_inches='tight')

In [ ]:
plt.figure()
plt.plot( gps[:,0], Alt )
plt.plot( timeStamp_selection, Alt_selection, "yD", markeredgecolor = 'black' )
plt.grid()
plt.title( "Altitude - " + data_date + " - files {} to {}".format(firstFile, lastFile) )

## Load attitude data logEkfEuler

In [ ]:
idx_timeStamp = 0
idx_roll = 1
idx_pitch = 2
idx_yaw = 3
idx_rollStdDev = 4
idx_pitchStdDev = 5
idx_yawStdDev = 6
idx_status = 7

In [ ]:
euler_timeStamp_13h00, roll_13h00, pitch_13h00, yaw_13h00 = np.loadtxt( logEkfEuler_13h00, 
                                                                       skiprows = 1, 
                                                                       usecols = (0, idx_roll, idx_pitch, idx_yaw), 
                                                                       unpack=True )
euler_timeStamp = euler_timeStamp_13h00
roll = roll_13h00
pitch = pitch_13h00
yaw = yaw_13h00

In [ ]:
euler_timeStamp[np.where(euler_timeStamp < 2**31)] +=  2**32

In [ ]:
# interpolate euler data for each file of the selection
roll_selection = np.interp( timeStamp_selection, euler_timeStamp, roll )
pitch_selection = np.interp( timeStamp_selection, euler_timeStamp, pitch )
yaw_selection = np.interp( timeStamp_selection, euler_timeStamp, yaw )

In [ ]:
if 1:
    plt.figure()
    ax = plt.subplot(211)
    plt.plot(euler_timeStamp, yaw)
    plt.subplot(212, sharex=ax)
    plt.plot(gps[:,0], gps[:,idx_gps_alt])

## Load  navigation data logEkfNav

In [ ]:
idx_timeStamp = 0
idx_velNorth = 1
idx_velEast = 2
idx_velDown = 3
idx_velNorth_StdDev = 4
idx_velEast_StdDev = 5
idx_velDown_StdDev = 6
idx_ekf_Lat = 7
idx_ekf_Long = 8
idx_ekf_Alt = 9
idx_ekf_undulation = 10
dx_ekf_Lat_StdDev = 11
idx_ekf_Long_StdDev = 12
idx_ekf_Alt_StdDev = 13
idx_status = 14

In [ ]:
nav_timeStamp_13h00, nav_lat_13h00, nav_long_13h00, nav_alt_13h00 = np.loadtxt( 
    logEkfNav_13h00, 
    skiprows = 1, 
    usecols = (0, idx_ekf_Lat, idx_ekf_Long, idx_ekf_Alt), 
    unpack = True )

nav_timeStamp = nav_timeStamp_13h00
nav_lat = nav_lat_13h00
nav_long = nav_long_13h00
nav_alt = nav_alt_13h00

In [ ]:
nav_timeStamp[np.where(nav_timeStamp < 2**31)] +=  2**32

### Compare gps and ekf

In [ ]:
withComparison = 1

In [ ]:
plt.figure()

ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax, Long, Lat )
plt.plot( nav_long, nav_lat, 'b' )
plotRunaway( ax )

plt.plot( Long_records, Lat_records, "Dg", markeredgecolor='black', 
         label=data_date )
plt.plot( Long_selection, Lat_selection, "Dy", markeredgecolor='black', 
         label="files selected for focalization {} to {}".format(firstFile, lastFile) )

title = "Navigation data-track references-record locations-" + data_date
plt.title("Navigation data, track references, record locations\n" + data_date)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.grid()

plt.savefig( data_dir + "/" + title + ".png", bbox_inches='tight')

In [ ]:
if withComparison:
    plt.figure()
    plt.plot( gps[:,0], Alt, label="gps" )
    plt.plot( nav_timeStamp, nav_alt, 'b', label="ekfNav" )
    plt.plot( timeStamp_selection, Alt_selection, "yD", markeredgecolor = 'black' )
    plt.grid()
    plt.title( "Altitude - " + data_date + " - files {} to {}".format(firstFile, lastFile) )
    plt.legend()

### Track 1

In [ ]:
files = "files {} to {}".format( firstFile, lastFile )

In [ ]:
plt.figure()

title = "altitude - velocity - course " + data_date

plt.subplot(311)
plt.title( title )
plt.plot( timeStamp_selection, Alt_selection, 'Dy', markeredgecolor = 'k', label="altitude " + files )
plt.legend()
plt.grid()

plt.subplot(312)
plt.plot( timeStamp_selection, Vel_selection, 'Dy', markeredgecolor = 'k', label="velocity " + files )
plt.legend()
plt.grid()

plt.subplot(313)
plt.plot( timeStamp_selection, course_selection, 'Dy', markeredgecolor = 'k', label="course " + files )
plt.legend()
plt.grid()

plt.savefig( data_dir + "/" + title + ".png", bbox_inches='tight')

In [ ]:
plt.figure()

title = "pitch - roll - yaw " + data_date

plt.subplot(311)
plt.title( title )
plt.plot( timeStamp_selection, pitch_selection, 'Dy', markeredgecolor = 'k', label="pitch " + files )
plt.legend()
plt.grid()

plt.subplot(312)
plt.plot( timeStamp_selection, roll_selection, 'Dy', markeredgecolor = 'k',  label="roll " + files )
plt.legend()
plt.grid()

plt.subplot(313)
plt.plot( timeStamp_selection, yaw_selection, 'Dy', markeredgecolor = 'k',  label="yaw " + files )
plt.legend()
plt.grid()

plt.savefig( data_dir + "/" + title + ".png", bbox_inches='tight')

In [ ]:
plt.figure()
ax = plt.subplot(111)
plotRunaway( ax )
plt.title( "track during the record of the files selection" )
plt.plot( Long_selection, Lat_selection )
plt.grid()

### Track linear approximation

In [ ]:
from posarmctools.track import *

In [ ]:
xdata, ydata = getxy( Lat_selection, Long_selection, startingPoint )

In [ ]:
popt, pcov = curve_fit(func, xdata, ydata)
a = popt[0]
b = popt[1]
print( "a = {}, b = {}".format( a, b ) )

In [ ]:
plt.figure()
plt.plot( xdata, ydata, label="track" )
plt.plot( xdata, func(xdata, a, b), label="linear approximation" )
plt.grid()
plt.legend()

In [ ]:
import json

In [ ]:
# unit vector
ux = ( 1, a ) / ( 1 + a**2 )**0.5
uy = [ -a, 1 ] / ( 1 + a**2 )**0.5
track_model = { "trackModel" : "model(x, p) = p[1]*x + p[2]",
               "p" : (a, b),
               "ux" : (-ux[0], -ux[1]),
               "uy" : (-uy[0], -uy[1]),
               "startingPointLatitude" : startingPoint[0],
               "startingPointLongitude" : startingPoint[1]
              }

In [ ]:
trackFilename = data_dir + "/track_model.json"
with open( trackFilename, 'w' ) as f:
    json.dump( track_model, f )

In [ ]:
# runaway projection
runaway_proj = np.zeros( runaway.shape )
getProjection( runaway, runaway_proj, startingPoint, track_model )
np.save( data_dir + "/runaway_proj", runaway_proj )
# hangar
hangar_proj = np.zeros( hangar.shape )
getProjection( hangar, hangar_proj, startingPoint, track_model )
np.save( data_dir + "/hangar_proj", hangar_proj )
# building
building_proj = np.zeros( building.shape )
getProjection( building, building_proj, startingPoint, track_model )
np.save( data_dir + "/building_proj", building_proj )
# corner reflector large
cornerReflectorLarge_proj = np.zeros( cornerReflectorLarge.shape )
getProjection( cornerReflectorLarge, cornerReflectorLarge_proj, startingPoint, track_model )
np.save( data_dir + "/cornerReflectorLarge_proj", cornerReflectorLarge_proj )
# corner reflector small
cornerReflectorSmall_proj = np.zeros( cornerReflectorSmall.shape )
getProjection( cornerReflectorSmall, cornerReflectorSmall_proj, startingPoint, track_model )
np.save( data_dir + "/cornerReflectorSmall_proj", cornerReflectorSmall_proj )

In [ ]:
track_proj = np.zeros( (Lat.shape[0],2) )
getProjection( np.stack( (Lat, Long), -1 ), track_proj, startingPoint, track_model )
np.save( data_dir + "/track_proj", track_proj )

In [ ]:
x_selection = np.interp( timeStamp_selection, gps[:,0], track_proj[:,0] )
y_selection = np.interp( timeStamp_selection, gps[:,0], track_proj[:,1] )
np.save( data_dir + "/track_selection_proj", np.stack( (x_selection, y_selection ), -1 ) )

In [ ]:
plt.figure()

plt.plot( runaway_proj[:,0], runaway_proj[:,1] )
plt.plot( hangar_proj[:,0], hangar_proj[:,1] )
plt.plot( building_proj[:,0], building_proj[:,1] )

plt.plot( track_proj[:,0], track_proj[:,1], "gray" )

plt.plot( x_selection, y_selection, 'Dy', markerEdgecolor='k',
        label = "files {} to {}".format( firstFile, lastFile ) )

plt.plot( cornerReflectorLarge_proj[0], cornerReflectorLarge_proj[1], 'o', color='white', markeredgecolor='k' )
plt.plot( cornerReflectorSmall_proj[0], cornerReflectorSmall_proj[1], 'o', color='white', markeredgecolor='k' )

ax = plt.gca()
ax.invert_xaxis()
ax.invert_yaxis()
ax.xaxis.tick_top()
ax.yaxis.tick_right()
ax.set_aspect(1)
plt.xlabel("location of the records in the scene coordinate system\n" + data_date)
plt.grid()
plt.legend()

## Interpolate navigation data for all ramps

### Build timestamp vector
Two options: numpy.interp vs scipy.interpolate.interp1d (with kind='linear')

In [ ]:
# rampNumber timeStamp x y z
rampNumber = np.arange( rampsPerFile * (nbRecords + 1) )

rampNumber_2 = np.concatenate( (rampNumber_1, [ rampNumber_1[-1] + rampsPerFile ] ) )
timeStamp_2 = np.concatenate( (timeStamp_1, [ timeStamp_1[-1] + timeStamp_1[-1] - timeStamp_1[-2] ] ) )

timeStamp = np.interp( rampNumber, rampNumber_2, timeStamp_2)
x = np.interp( timeStamp, gps[:,0], track_proj[:,0] )
y = np.interp( timeStamp, gps[:,0], track_proj[:,1] )
z = np.interp( timeStamp, gps[:,0], gps[:,3] )
xyz_proj_allRamps = np.stack( (rampNumber, timeStamp, x, y, z), -1 )

In [ ]:
np.save( data_dir + "/rampNumber_timeStamp_xyz", xyz_proj_allRamps )

## Build data for rampUp

In [ ]:
# rampNumber timeStamp x y z

timeStamp_2 = np.concatenate( (timeStamp_1, [ timeStamp_1[-1] + timeStamp_1[-1] - timeStamp_1[-2] ] ) ) \
    + params.configuredTRamp / 2

timeStamp = np.interp( rampNumber, rampNumber_2, timeStamp_2)
x = np.interp( timeStamp, gps[:,0], track_proj[:,0] )
y = np.interp( timeStamp, gps[:,0], track_proj[:,1] )
z = np.interp( timeStamp, gps[:,0], gps[:,3] )
xyz_proj_allRamps = np.stack( (rampNumber, timeStamp, x, y, z), -1 )

In [ ]:
np.save( data_dir + "/rampNumber_timeStamp_xyz_rampUp", xyz_proj_allRamps )

In [ ]:
plt.figure()
plt.plot( x_selection, y_selection, 'Dy', markerEdgecolor='k', label = "files {} to {}".format( firstFile, lastFile ) )
firstRamp = firstFile * rampsPerFile
lastRamp = firstRamp + (nbFiles+1) * rampsPerFile
plt.plot( x[firstRamp : lastRamp], y[firstRamp : lastRamp], '.b', label = 'ramps' )
plt.legend()
plt.title("xy")
plt.grid()

In [ ]:
plt.figure()
plt.plot(rampNumber_1, timeStamp_1)
plt.plot(rampNumber_2, timeStamp_2, 'o')